In [1]:
!nvidia-smi

Fri Sep 20 19:35:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Introdução a Operações Customizadas

---




In [2]:
%%writefile saxpy.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <iostream>
#include <cstdlib>  // Para gerar números aleatórios
#include <chrono>   // Para medir o tempo de execução

// Functor Saxpy: realiza a operação z = a * x + y
struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {}

    __host__ __device__
    double operator()(const double& x, const double& y) const {
        return a * x + y;
    }
};

int main() {
    // Parâmetros
    int N = 1000000;  // Tamanho dos vetores
    int a = 5;        // Constante 'a' no cálculo Saxpy

    // Medir tempo de execução
    auto start = std::chrono::high_resolution_clock::now();

    // Vetores de entrada na CPU (host)
    thrust::host_vector<double> h_x(N);
    thrust::host_vector<double> h_y(N);

    // Preenche os vetores com valores aleatórios
    for (int i = 0; i < N; i++) {
        h_x[i] = static_cast<double>(rand()) / RAND_MAX;  // Números aleatórios entre 0 e 1
        h_y[i] = static_cast<double>(rand()) / RAND_MAX;
    }

    // Transferir os vetores para a GPU (device)
    thrust::device_vector<double> d_x = h_x;
    thrust::device_vector<double> d_y = h_y;
    thrust::device_vector<double> d_z(N);  // Vetor de saída

    // Aplicar a operação Saxpy
    thrust::transform(d_x.begin(), d_x.end(), d_y.begin(), d_z.begin(), saxpy(a));

    // Transferir o resultado de volta para a CPU
    thrust::host_vector<double> h_z = d_z;

    // Medir o fim da execução
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end - start;

    // Exibir os primeiros 5 resultados para verificação
    std::cout << "Primeiros 5 resultados do cálculo Saxpy (a = " << a << "):" << std::endl;
    for (int i = 0; i < 5; i++) {
        std::cout << "z[" << i << "] = " << h_z[i] << std::endl;
    }

    // Exibir o tempo de execução
    std::cout << "Tempo de execução: " << elapsed.count() << " segundos" << std::endl;

    return 0;
}

Writing saxpy.cu


In [3]:
!nvcc -arch=sm_75 saxpy.cu -o saxpy

In [4]:
!./saxpy

Primeiros 5 resultados do cálculo Saxpy (a = 5):
z[0] = 4.59532
z[1] = 4.71394
z[2] = 4.75579
z[3] = 2.44434
z[4] = 1.94284
Tempo de execução: 0.462703 segundos


### Transformações Unárias e Binárias


Implementação com CUDA/Thrust (GPU)

In [23]:
%%writefile magnitude.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <cmath>   // Para sqrt
#include <iostream>
#include <cstdlib> // Para gerar números aleatórios
#include <chrono>  // Para medir o tempo de execução

// Functor para elevar ao quadrado
struct square {
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;
    }
};

// Função para calcular a magnitude de um vetor na GPU
float magnitude_gpu(thrust::device_vector<float>& v) {
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);
}

int main() {
    int N = 1000000;  // Tamanho do vetor

    // Medir tempo de execução na GPU
    auto start_gpu = std::chrono::high_resolution_clock::now();

    // Criar vetor na CPU e preenchê-lo com números aleatórios
    thrust::host_vector<float> h_v(N);
    for (int i = 0; i < N; i++) {
        h_v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Transferir o vetor para a GPU
    thrust::device_vector<float> d_v = h_v;

    // Calcular a magnitude na GPU
    float result_gpu = magnitude_gpu(d_v);

    // Medir o tempo de execução na GPU
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_gpu = end_gpu - start_gpu;

    // Exibir o resultado e o tempo de execução na GPU
    std::cout << "Magnitude calculada na GPU: " << result_gpu << std::endl;
    std::cout << "Tempo de execução na GPU: " << elapsed_gpu.count() << " segundos" << std::endl;

    return 0;
}


Overwriting magnitude.cu


Implementação em C++ (CPU)

In [24]:
%%writefile magnitude_cpu.cpp
#include <iostream>
#include <vector>
#include <cmath>   // Para sqrt
#include <cstdlib> // Para gerar números aleatórios
#include <chrono>  // Para medir o tempo de execução

// Função para calcular a magnitude de um vetor na CPU
float magnitude_cpu(const std::vector<float>& v) {
    float sum_of_squares = 0.0f;
    for (size_t i = 0; i < v.size(); i++) {
        sum_of_squares += v[i] * v[i];
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    int N = 1000000;  // Tamanho do vetor

    // Medir tempo de execução na CPU
    auto start_cpu = std::chrono::high_resolution_clock::now();

    // Criar vetor e preenchê-lo com números aleatórios
    std::vector<float> v(N);
    for (int i = 0; i < N; i++) {
        v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Calcular a magnitude na CPU
    float result_cpu = magnitude_cpu(v);

    // Medir o tempo de execução na CPU
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_cpu = end_cpu - start_cpu;

    // Exibir o resultado e o tempo de execução na CPU
    std::cout << "Magnitude calculada na CPU: " << result_cpu << std::endl;
    std::cout << "Tempo de execução na CPU: " << elapsed_cpu.count() << " segundos" << std::endl;

    return 0;
}


Overwriting magnitude_cpu.cpp


In [25]:
!nvcc -arch=sm_75 magnitude.cu -o magnitude_gpu

In [26]:
!g++ magnitude_cpu.cpp -o magnitude_cpu

In [27]:
!./magnitude_gpu

Magnitude calculada na GPU: 577.328
Tempo de execução na GPU: 0.305413 segundos


In [28]:
!./magnitude_cpu

Magnitude calculada na CPU: 577.228
Tempo de execução na CPU: 0.037438 segundos


### Otimização com Fusion Kernel


### Implementação com Fusion Kernel (GPU)

In [36]:
%%writefile variance_fusion.cu
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/host_vector.h>   // Corrigido: Inclui o cabeçalho correto para host_vector
#include <iostream>
#include <cmath>   // Para sqrt
#include <cstdlib> // Para gerar números aleatórios
#include <chrono>  // Para medir o tempo de execução

// Functor para calcular a diferença ao quadrado
struct variance_op {
    float mean;
    variance_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Função para calcular a variância com Fusion Kernel
float calculate_variance_fusion(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

int main() {
    int N = 1000000;  // Tamanho do vetor

    // Criar vetor na CPU e preenchê-lo com números aleatórios
    thrust::host_vector<float> h_vec(N);  // Agora corretamente utilizando thrust::host_vector
    for (int i = 0; i < N; i++) {
        h_vec[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Transferir o vetor para a GPU
    thrust::device_vector<float> d_vec = h_vec;

    // Calcular a média primeiro
    float sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>());
    float mean = sum / N;

    // Medir tempo de execução para o cálculo da variância com Fusion Kernel
    auto start_fusion = std::chrono::high_resolution_clock::now();

    // Calcular a variância usando Fusion Kernel
    float variance_fusion = calculate_variance_fusion(d_vec, mean);

    // Medir o tempo de execução
    auto end_fusion = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_fusion = end_fusion - start_fusion;

    // Exibir o resultado da variância e o tempo de execução
    std::cout << "Variância calculada com Fusion Kernel: " << variance_fusion << std::endl;
    std::cout << "Tempo de execução (Fusion Kernel): " << elapsed_fusion.count() << " segundos" << std::endl;

    return 0;
}


Overwriting variance_fusion.cu


In [37]:
!nvcc -arch=sm_75 variance_fusion.cu -o variance_fusion

In [38]:
!./variance_fusion

Variância calculada com Fusion Kernel: 0.0833013
Tempo de execução (Fusion Kernel): 0.000406248 segundos


In [39]:
%%writefile variance_no_fusion.cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>       // Para thrust::transform
#include <thrust/reduce.h>          // Para thrust::reduce
#include <thrust/host_vector.h>      // Inclui o cabeçalho correto para host_vector
#include <iostream>
#include <cmath>   // Para sqrt
#include <cstdlib> // Para gerar números aleatórios
#include <chrono>  // Para medir o tempo de execução

// Functor para calcular a diferença ao quadrado
struct square_diff {
    float mean;
    square_diff(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Função para calcular a variância sem Fusion Kernel
float calculate_variance_no_fusion(const thrust::device_vector<float>& d_vec, float mean) {
    // Etapa 1: Calcular a soma das diferenças ao quadrado
    thrust::device_vector<float> d_diff(d_vec.size());
    thrust::transform(d_vec.begin(), d_vec.end(), d_diff.begin(), square_diff(mean));

    // Etapa 2: Reduzir a soma das diferenças ao quadrado
    float sum_of_squares = thrust::reduce(d_diff.begin(), d_diff.end(), 0.0f, thrust::plus<float>());
    return sum_of_squares / d_vec.size();
}

int main() {
    int N = 1000000;  // Tamanho do vetor

    // Criar vetor na CPU e preenchê-lo com números aleatórios
    thrust::host_vector<float> h_vec(N);  // Corrigido: Inclui host_vector corretamente
    for (int i = 0; i < N; i++) {
        h_vec[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Transferir o vetor para a GPU
    thrust::device_vector<float> d_vec = h_vec;

    // Calcular a média primeiro
    float sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>());
    float mean = sum / N;

    // Medir tempo de execução para o cálculo da variância sem Fusion Kernel
    auto start_no_fusion = std::chrono::high_resolution_clock::now();

    // Calcular a variância sem Fusion Kernel
    float variance_no_fusion = calculate_variance_no_fusion(d_vec, mean);

    // Medir o tempo de execução
    auto end_no_fusion = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_no_fusion = end_no_fusion - start_no_fusion;

    // Exibir o resultado da variância e o tempo de execução
    std::cout << "Variância calculada sem Fusion Kernel: " << variance_no_fusion << std::endl;
    std::cout << "Tempo de execução (sem Fusion Kernel): " << elapsed_no_fusion.count() << " segundos" << std::endl;

    return 0;
}


Overwriting variance_no_fusion.cu


In [40]:
!nvcc -arch=sm_75 variance_no_fusion.cu -o variance_no_fusion

In [41]:
!./variance_no_fusion

Variância calculada sem Fusion Kernel: 0.0833013
Tempo de execução (sem Fusion Kernel): 0.000689659 segundos
